*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.3
IPython 6.2.1

torch 0.3.0.post4


# Model Zoo -- Multilayer Perceptron

In [2]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import numpy as np


##########################
### SETTINGS
##########################

# Hyperparameters
learning_rate = 0.1
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [3]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        # The following to lones are not necessary, 
        # but used here to demonstrate how to access the weights
        # and use a different weight initialization.
        # By default, PyTorch uses Xavier/Glorot initialization, which
        # should usually be preferred.
        self.linear_1.weight.data.normal_(0.0, 0.1)
        self.linear_1.bias.data.zero_()
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.data.normal_(0.0, 0.1)
        self.linear_2.bias.data.normal_(0.0, 0.01)
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.data.normal_(0.0, 0.1)
        self.linear_out.bias.data.normal_(0.0, 0.1)
        
    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(0)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

if torch.cuda.is_available():
    model.cuda()
    

##########################
### COST AND OPTIMIZER
##########################

cost_fn = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [4]:
from torch.autograd import Variable
import torch.nn.functional as F

def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = Variable(features.view(-1, 28*28))
        if torch.cuda.is_available():
            features = features.cuda()
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas.data, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels.cpu() == targets).sum()
    return correct_pred/num_examples * 100
    

for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = Variable(features.view(-1, 28*28))
        targets = Variable(targets)
        
        if torch.cuda.is_available():
            features, targets = features.cuda(), targets.cuda()
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost.data[0]))

    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/937 | Cost: 2.6257
Epoch: 001/010 | Batch 050/937 | Cost: 0.5718
Epoch: 001/010 | Batch 100/937 | Cost: 0.5704
Epoch: 001/010 | Batch 150/937 | Cost: 0.3470
Epoch: 001/010 | Batch 200/937 | Cost: 0.2210
Epoch: 001/010 | Batch 250/937 | Cost: 0.4168
Epoch: 001/010 | Batch 300/937 | Cost: 0.2414
Epoch: 001/010 | Batch 350/937 | Cost: 0.2745
Epoch: 001/010 | Batch 400/937 | Cost: 0.5059
Epoch: 001/010 | Batch 450/937 | Cost: 0.4488
Epoch: 001/010 | Batch 500/937 | Cost: 0.2738
Epoch: 001/010 | Batch 550/937 | Cost: 0.1451
Epoch: 001/010 | Batch 600/937 | Cost: 0.1684
Epoch: 001/010 | Batch 650/937 | Cost: 0.1255
Epoch: 001/010 | Batch 700/937 | Cost: 0.2672
Epoch: 001/010 | Batch 750/937 | Cost: 0.1492
Epoch: 001/010 | Batch 800/937 | Cost: 0.2204
Epoch: 001/010 | Batch 850/937 | Cost: 0.2064
Epoch: 001/010 | Batch 900/937 | Cost: 0.2630
Epoch: 001/010 training accuracy: 94.80%
Epoch: 002/010 | Batch 000/937 | Cost: 0.1440
Epoch: 002/010 | Batch 050/937 | Cost: 

Epoch: 009/010 training accuracy: 99.25%
Epoch: 010/010 | Batch 000/937 | Cost: 0.0514
Epoch: 010/010 | Batch 050/937 | Cost: 0.0362
Epoch: 010/010 | Batch 100/937 | Cost: 0.0268
Epoch: 010/010 | Batch 150/937 | Cost: 0.0207
Epoch: 010/010 | Batch 200/937 | Cost: 0.0051
Epoch: 010/010 | Batch 250/937 | Cost: 0.0238
Epoch: 010/010 | Batch 300/937 | Cost: 0.0017
Epoch: 010/010 | Batch 350/937 | Cost: 0.0350
Epoch: 010/010 | Batch 400/937 | Cost: 0.0214
Epoch: 010/010 | Batch 450/937 | Cost: 0.0157
Epoch: 010/010 | Batch 500/937 | Cost: 0.0681
Epoch: 010/010 | Batch 550/937 | Cost: 0.0322
Epoch: 010/010 | Batch 600/937 | Cost: 0.0460
Epoch: 010/010 | Batch 650/937 | Cost: 0.0120
Epoch: 010/010 | Batch 700/937 | Cost: 0.0098
Epoch: 010/010 | Batch 750/937 | Cost: 0.0028
Epoch: 010/010 | Batch 800/937 | Cost: 0.0718
Epoch: 010/010 | Batch 850/937 | Cost: 0.0599
Epoch: 010/010 | Batch 900/937 | Cost: 0.0746
Epoch: 010/010 training accuracy: 99.23%


In [5]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.75%
